In [1]:
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ee.Authenticate()

ee.Initialize(project='general-porpose')

geometry = ee.Geometry.Polygon(
    [[[-56.19933935093308, -15.570030087161426],
      [-56.236418208354955, -15.63616420965171],
      [-56.173246821636205, -15.75515175246836],
      [-56.110075434917455, -15.760438474073672],
      [-56.079863032573705, -15.701615952093842],
      [-56.002958735698705, -15.706243058621192],
      [-55.91712804722214, -15.675173328966476],
      [-55.92468114780808, -15.622277812891932],
      [-55.98510595249558, -15.587888400489534],
      [-56.00982519077683, -15.527031492578372],
      [-56.052397212261205, -15.503213026342152],
      [-56.09290929722214, -15.518430696875168],
      [-56.13616796421433, -15.537616594867629]]])


ModuleNotFoundError: No module named 'StringIO'

### Sentinela

In [20]:
def mask_clouds_sentinel(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10  # clouds
    cirrus_bit_mask = 1 << 11  # cirrus
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).copyProperties(image, image.propertyNames())

def analisar_indice_numpy(array, threshold):
    validos = ~np.isnan(array)
    binario = (array > threshold) & validos
    acima = np.count_nonzero(binario)
    abaixo = np.count_nonzero((~binario) & validos)
    total = acima + abaixo
    proporcao = acima / total if total > 0 else None
    return {
        'acima': acima,
        'abaixo': abaixo,
        'proporcao': proporcao
    }

anos = range(2016, 2025)
meses = range(7, 10)
resultados = []

threshold_map = {'NDVI': 0.15, 'EVI': 0.5, 'NDBI': 0.01}
escala = 20  # Sentinel-2 tem 10m de resolução para B2, B4, B8

for ano in anos:
    for mes in meses:
        try:
            inicio = f"{ano}-{mes:02d}-01"
            fim = f"{ano}-{mes:02d}-28"

            colecao = ee.ImageCollection("COPERNICUS/S2_SR") \
                .filterBounds(geometry) \
                .filterDate(inicio, fim) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 70)) \
                .map(mask_clouds_sentinel)

            imagem = colecao.sort('CLOUDY_PIXEL_PERCENTAGE').first()

            # Testa se há imagem
            if imagem is None or imagem.getInfo() is None:
                continue

            nir = imagem.select('B8')
            red = imagem.select('B4')
            blue = imagem.select('B2')
            swir = imagem.select('B11')

            ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
            evi = imagem.expression(
                '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
                {'NIR': nir, 'RED': red, 'BLUE': blue}
            ).rename('EVI')
            ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI')

            imagem = imagem.addBands([ndvi, evi, ndbi])

            res = {}
            aux = []
            for indice in ['NDVI', 'EVI']:
                banda = imagem.select(indice)
                array = geemap.ee_to_numpy(banda, region=geometry, scale=escala)
                array = np.squeeze(array)

                index_res = analisar_indice_numpy(array, threshold=threshold_map[indice])
                res.update({f"{k}_{indice}": v for k, v in index_res.items()})
                aux.append(index_res['proporcao'])

            resultados.append({
                'ano': ano,
                'mes': mes,
                'data': imagem.date().format("YYYY-MM-dd").getInfo(),
                'cloud_percent': imagem.get('CLOUDY_PIXEL_PERCENTAGE').getInfo(),
                **res,
                'image_id': imagem.get('system:id').getInfo()
            })
            print(f"{ano}-{mes:02d} => NDVI = {aux[0]:.2f} EVI = {aux[1]:.2f}")

        except Exception as e:
            print(f"Erro Sentinel-2 {ano}-{mes:02d}: {e}")

df_sentinel = pd.DataFrame(resultados)
df_sentinel.to_csv("sentinel_ndvi_mensal_metadata.csv", index=False)
print("CSV do Sentinel-2 salvo.")


2019-07 => NDVI = 0.72 EVI = 0.62
2019-08 => NDVI = 0.42 EVI = 0.35
2019-09 => NDVI = 0.41 EVI = 0.28
2020-07 => NDVI = 0.42 EVI = 0.37
2020-08 => NDVI = 0.40 EVI = 0.25
2020-09 => NDVI = 0.38 EVI = 0.29
2021-07 => NDVI = 0.42 EVI = 0.31
2021-08 => NDVI = 0.40 EVI = 0.18
2021-09 => NDVI = 0.37 EVI = 0.14
2022-07 => NDVI = 0.39 EVI = 0.38
2022-08 => NDVI = 0.62 EVI = 0.65
2022-09 => NDVI = 0.59 EVI = 0.59
2023-07 => NDVI = 0.38 EVI = 0.38
2023-08 => NDVI = 0.59 EVI = 0.56
Erro Sentinel-2 2023-09: Collection.first: Error in map(ID=20230903T135711_20230903T140112_T21LWC):
Image.select: Band pattern 'QA60' did not match any bands. Available bands: [B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B11, B12, AOT, WVP, SCL, TCI_R, TCI_G, TCI_B, MSK_CLDPRB, MSK_SNWPRB, MSK_CLASSI_OPAQUE, MSK_CLASSI_CIRRUS, MSK_CLASSI_SNOW_ICE]
Erro Sentinel-2 2024-07: Collection.first: Error in map(ID=20240724T135709_20240724T135955_T21LXC):
Image.select: Band pattern 'QA60' did not match any bands. Available bands: [

### Landsat

In [ ]:
def mask_clouds(image):
    qa = image.select('QA_PIXEL')
    cloud_bit = 1 << 3
    shadow_bit = 1 << 4
    mask = qa.bitwiseAnd(cloud_bit).eq(0).And(qa.bitwiseAnd(shadow_bit).eq(0))
    return image.updateMask(mask)


def analisar_indice_numpy(array, threshold):
    validos = ~np.isnan(array)
    binario = (array > threshold) & validos
    acima = np.count_nonzero(binario)
    abaixo = np.count_nonzero((~binario) & validos)
    total = acima + abaixo
    proporcao = acima / total if total > 0 else None
    # media = np.nanmean(array)
    return {
        'acima': acima,
        'abaixo': abaixo,
        'proporcao': proporcao
    }

anos = range(2013, 2025)
meses = range(7, 10)
resultados = []

bandas = {
        'LANDSAT_8':  {'NIR': 'SR_B5', 'RED': 'SR_B4', 'BLUE': 'SR_B2', 'SWIR': 'SR_B6'},
        'LANDSAT_9':  {'NIR': 'SR_B5', 'RED': 'SR_B4', 'BLUE': 'SR_B2', 'SWIR': 'SR_B6'},
        'default':  {'NIR': 'SR_B5', 'RED': 'SR_B4', 'BLUE': 'SR_B2', 'SWIR': 'SR_B6'},

    }

threshold_map = {'NDVI': 0.15,'EVI': 0.5,'NDBI': 0.01}
escala = 15


def get_band(bands_dict, band_name,sensor):
    return bands_dict.get(sensor, bands_dict['default'])[band_name]

colecao_merged = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .merge(ee.ImageCollection("LANDSAT/LC09/C02/T1_L2"))

for ano in anos:
    for mes in meses:
        try:
            inicio = f"{ano}-{mes:02d}-01"
            fim = f"{ano}-{mes:02d}-28"

            # colecao = colecao_merged \
            #     .filterBounds(geometry) \
            #     .filterDate(inicio, fim) \
            #     .filter(ee.Filter.lt('CLOUD_COVER', 70)) \
            #     .map(mask_clouds) \
            #     .map(process_image)
            
            colecao = colecao_merged \
                .filterBounds(geometry) \
                .filterDate(inicio, fim) \
                .filter(ee.Filter.lt('CLOUD_COVER', 70))\
                .map(mask_clouds)
                
            imagem = colecao.sort('CLOUD_COVER').first()
            sensor = ee.String(imagem.get('SPACECRAFT_ID')).getInfo()
            
            # Testa se há imagem
            if imagem is None or imagem.getInfo() is None:
                continue

            # Extrai NDVI como array NumPy
            nir = imagem.select(get_band(bandas, 'NIR',sensor))
            red = imagem.select(get_band(bandas, 'RED',sensor))
            blue = imagem.select(get_band(bandas, 'BLUE',sensor))
            swir = imagem.select(get_band(bandas, 'SWIR',sensor))
            ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
            evi = imagem.expression(
                '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
                {'NIR': nir, 'RED': red, 'BLUE': blue}
            ).rename('EVI')
            ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI')

            imagem = imagem.addBands([ndvi, evi, ndbi])

            res = {}
            aux = []
            for indice in ['NDVI', 'EVI']:
                banda = imagem.select(indice)
                array = geemap.ee_to_numpy(banda, region=geometry, scale=escala)
                array = np.squeeze(array) 
                        
                index_res = analisar_indice_numpy(array, threshold=threshold_map[indice])
                res.update({f"{k}_{indice}":v for k,v in index_res.items()})
                aux.append(index_res['proporcao'])

            resultados.append({
                'ano': ano,
                'mes': mes,
                'data': imagem.date().format("YYYY-MM-dd").getInfo(),
                'cloud_percent': imagem.get('CLOUD_COVER').getInfo(),
                **res,
                'image_id': imagem.get('system:id').getInfo()
            })
            print(f"{ano}-{mes:02d} => NDVI = {aux[0]:.2f} EVI = {aux[1]:.2f}")

        except Exception as e:
            print(f"Erro Landsat {ano}-{mes:02d}: {e}")

df_landsat = pd.DataFrame(resultados)
df_landsat.to_csv("landsat_ndvi_mensal_metadata.csv", index=False)
print("CSV do Landsat salvo.")



2013-07 => NDVI = 0.80 EVI = 0.88
2013-08 => NDVI = 0.68 EVI = 0.80
2013-09 => NDVI = 0.57 EVI = 0.68
2014-07 => NDVI = 0.82 EVI = 0.88
2014-08 => NDVI = 0.78 EVI = 0.86
2014-09 => NDVI = 0.81 EVI = 0.90
Erro Landsat 2015-07: Element.get: Parameter 'object' is required and may not be null.
2015-08 => NDVI = 0.76 EVI = 0.85
2015-09 => NDVI = 0.77 EVI = 0.87
2016-07 => NDVI = 0.73 EVI = 0.84
2016-08 => NDVI = 0.70 EVI = 0.87
2016-09 => NDVI = 0.72 EVI = 0.87
2017-07 => NDVI = 0.78 EVI = 0.86
2017-08 => NDVI = 0.70 EVI = 0.81
2017-09 => NDVI = 0.75 EVI = 0.87
2018-07 => NDVI = 0.75 EVI = 0.86
2018-08 => NDVI = 0.68 EVI = 0.73
Erro Landsat 2018-09: Element.get: Parameter 'object' is required and may not be null.
2019-07 => NDVI = 0.79 EVI = 0.87
2019-08 => NDVI = 0.70 EVI = 0.82
2019-09 => NDVI = 0.72 EVI = 0.83
2020-07 => NDVI = 0.73 EVI = 0.82
2020-08 => NDVI = 0.69 EVI = 0.79
2020-09 => NDVI = 0.57 EVI = 0.75
2021-07 => NDVI = 0.71 EVI = 0.82
2021-08 => NDVI = 0.55 EVI = 0.67
2021-09 =>

### Com Limiar Ajustado

In [ ]:
def lista_imagens_evi(geometria, escala, sensor):
    if sensor == 'LANDSAT':
        colecao = ee.ImageCollection("COPERNICUS/S2_SR") \
                .filterBounds(geometry) \
                .filterDate(inicio, fim) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 70)) \
                .map(mask_clouds_sentinel)

        imagem = colecao.sort('CLOUDY_PIXEL_PERCENTAGE').first()


def limiar_adaptativo_com_crescimento(imagens_evi, threshold_inicial):
    """
    imagens_evi: lista de arrays 2D (EVI) ordenadas no tempo
    threshold_inicial: valor inicial de corte para a primeira imagem
    """
    resultados = []
    mascara_anterior = None
    threshold_anterior = threshold_inicial
    media_anterior = None

    for i, evi in enumerate(imagens_evi):
        evi = np.squeeze(evi)
        mascara = np.zeros_like(evi, dtype=bool)

        if i == 0:
            # Primeira imagem: aplica threshold inicial
            mascara = evi > threshold_inicial
            media_anterior = np.nanmean(evi[mascara])
            threshold_atual = threshold_inicial
        else:
            # Aplica a máscara anterior para obter os mesmos pixels
            valores_anteriores = evi[mascara_anterior]
            media_atual = np.nanmean(valores_anteriores)

            # Ajuste proporcional do limiar
            if media_atual is not None and media_anterior is not None and media_anterior > 0:
                threshold_atual = threshold_anterior * (media_atual / media_anterior)
            else:
                threshold_atual = threshold_anterior  # fallback

            # Aplica novo limiar
            mascara = evi > threshold_atual

            # Garante que todos os pixels anteriores permaneçam ativos
            mascara = np.logical_or(mascara, mascara_anterior)

            # Atualiza média para próxima iteração
            media_anterior = np.nanmean(evi[mascara])

        resultados.append({
            'evi_array': evi,
            'threshold': threshold_atual,
            'mascara': mascara
        })

        # Atualiza variáveis para próxima rodada
        mascara_anterior = mascara.copy()
        threshold_anterior = threshold_atual

    return resultados


In [ ]:
df_landsat.groupby('ano')['proporcao_EVI'].mean().plot(kind='bar', color='green', alpha=0.7)

In [ ]:
df_landsat[df_landsat['mes'].isin([8,9])].groupby('ano').agg({'proporcao_vegetado': 'mean'})

In [16]:
df_landsat[df_landsat['mes'].isin([9])]['proporcao_EVI']

2     0.683019
5     0.895129
7     0.870253
10    0.868776
13    0.873873
18    0.831147
21    0.746401
24    0.672773
27    0.857406
30    0.873892
33    0.704862
Name: proporcao_EVI, dtype: float64

In [ ]:
colecao = colecao_merged \
                .filterBounds(geometry) \
                .filterDate("2022-01-01", "2022-02-01") \
                .filter(ee.Filter.lt('CLOUD_COVER', 70)) \
                .map(mask_clouds) \
                .map(process_image)

imagem = colecao.sort('CLOUD_COVER').first()
imagem

In [ ]:
sensor = imagem.get('SPACECRAFT_ID').getInfo()

# Bandas RGB por sensor
bandas_rgb = {
    'LANDSAT_5': ['SR_B3', 'SR_B2', 'SR_B1'],  # R, G, B
    'LANDSAT_7': ['SR_B3', 'SR_B2', 'SR_B1'],
    'LANDSAT_8': ['SR_B4', 'SR_B3', 'SR_B2'],
    'LANDSAT_9': ['SR_B4', 'SR_B3', 'SR_B2']
}

rgb_bands = bandas_rgb.get(sensor, ['SR_B4', 'SR_B3', 'SR_B2']) 

In [ ]:
rgb_image = imagem.select(rgb_bands)
# region = geometry.centroid().buffer(300).bounds()  # recorte de 3 km

# Reduz escala se necessário (para não dar erro de tamanho)
rgb_array = geemap.ee_to_numpy(rgb_image, region=geometry, scale=100) 
rgb_array = np.clip(rgb_array / 30000, 0, 1)  # faixa típica Landsat

plt.figure(figsize=(8, 6))
plt.imshow(rgb_array)
plt.title(f'Imagem RGB - {sensor}')
plt.axis('off')
plt.show()


### Processa imagem a imagem

In [ ]:
df_landsat[df_landsat['mes']==7]

In [ ]:

def analisar_indice_numpy(array, threshold=0.15):
    validos = ~np.isnan(array)
    binario = (array > threshold) & validos
    acima = np.count_nonzero(binario)
    abaixo = np.count_nonzero((~binario) & validos)
    total = acima + abaixo
    proporcao = acima / total if total > 0 else None
    media = np.nanmean(array)
    return {
        'media': media,
        'pixels_acima_threshold': acima,
        'pixels_abaixo_threshold': abaixo,
        'proporcao_acima': proporcao
    }

def processar_indices(image_id, geometry, scale=20):
    image = ee.Image(image_id)
    
    # Detecta o sensor
    sensor = ee.String(image.get('SPACECRAFT_ID')).getInfo()
    threshold_map = {'NDVI': 0.15,'EVI': 0.5,'NDBI': 0.01}
    # Bandas corretas por sensor
    bandas = {
        'LANDSAT_8':  {'NIR': 'SR_B5', 'RED': 'SR_B4', 'BLUE': 'SR_B2', 'SWIR': 'SR_B6'},
        'LANDSAT_9':  {'NIR': 'SR_B5', 'RED': 'SR_B4', 'BLUE': 'SR_B2', 'SWIR': 'SR_B6'},
        'default':  {'NIR': 'SR_B5', 'RED': 'SR_B4', 'BLUE': 'SR_B2', 'SWIR': 'SR_B6'},

    }

    def get_band(bands_dict, band_name):
        return bands_dict.get(sensor, bands_dict['default'])[band_name]

    nir = image.select(get_band(bandas, 'NIR'))
    red = image.select(get_band(bandas, 'RED'))
    blue = image.select(get_band(bandas, 'BLUE'))
    swir = image.select(get_band(bandas, 'SWIR'))

    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {'NIR': nir, 'RED': red, 'BLUE': blue}
    ).rename('EVI')
    ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI')

    image = image.addBands([ndvi, evi, ndbi])

    resultados = {}
    imagensNP = {}
    for indice in ['NDVI', 'EVI', 'NDBI']:
        banda = image.select(indice)
        array = geemap.ee_to_numpy(banda, region=geometry, scale=scale)
        array = np.squeeze(array)  # Remove dimensões extras, se houver

        resultados[indice] = analisar_indice_numpy(array, threshold=threshold_map[indice])
        imagensNP[indice] = array


    return resultados,imagensNP



In [ ]:
aa,bb=processar_indices('LANDSAT/LC08/C02/T1_L2/LC08_226071_20130716', geometry)
aa1,bb1=processar_indices('LANDSAT/LC08/C02/T1_L2/LC08_226071_20190701', geometry)


In [ ]:
chave = 'EVI'

fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 linha, 2 colunas

# Imagem binária (NDVI > 0.5)
axes[0].imshow(bb1[chave] > 0.50, cmap='gray')
axes[0].set_title('NDVI > 0.5')
axes[0].axis('off')

# Imagem NDVI colorida
axes[1].imshow(bb1[chave], cmap='RdYlGn', vmin=-1, vmax=1)
axes[1].set_title('NDVI (escala)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 linha, 2 colunas

# Imagem binária (NDVI > 0.5)
axes[0].imshow(bb[chave] > 0.4597476565925763, cmap='gray')
axes[0].set_title('NDVI > 0.5')
axes[0].axis('off')

# Imagem NDVI colorida
axes[1].imshow(bb[chave], cmap='RdYlGn', vmin=-1, vmax=1)
axes[1].set_title('NDVI (escala)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
mbb = np.mean(bb[chave][bb[chave]>0.5])
mbb1 = np.mean(bb1[chave][bb[chave]>0.5])
print(mbb,mbb1)

In [ ]:
(mbb1/mbb)*0.5

In [ ]:
np.count_nonzero(bb1[chave] > (mbb1/mbb)*0.5)

In [ ]:
aa1['EVI']

In [ ]:
aa['EVI']

In [ ]:
analisar_ndvi_numpy(bb[chave], threshold=0.5)

In [ ]:
mbb = np.median(bb[chave][bb[chave]>0.5])
mbb1 = np.median(bb1[chave][bb[chave]>0.5])
print(mbb,mbb1)